In [1]:
!pip install transformers datasets torch sklearn

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

# Load dataset
df = pd.read_csv('/content/all_rows (2) (1).csv')
# Load datasets
# df_all_rows = pd.read_csv('/content/all_rows (2) (1).csv')
# df_food_data = pd.read_csv('/content/FoodData (1).csv')

# Preprocess text
df["ingredients_text"] = df["ingredients_text"].str.lower()

# Check for the actual column name containing allergy information
# Print the columns of your DataFrame to see the available columns
print(df.columns)

# Assuming the column containing allergy information is named 'allergens', replace 'allergens' with the actual column name
# Encode labels
label_encoder = LabelEncoder()
df["Allergy"] = label_encoder.fit_transform(df["allergens"]) # Change 'allergens' to the actual column name

# Split data into train and test sets
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)



Index(['allergens', 'ingredients_text', 'additives_en'], dtype='object')


In [5]:
import torch

In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(texts, labels, max_length=128):
    return tokenizer(
        texts.tolist(),
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    ), torch.tensor(labels.tolist())

train_encodings, train_labels = tokenize_data(train_df["ingredients_text"], train_df["Allergy"])
test_encodings, test_labels = tokenize_data(test_df["ingredients_text"], test_df["Allergy"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
from torch.utils.data import Dataset

class AllergyDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = AllergyDataset(train_encodings, train_labels)
test_dataset = AllergyDataset(test_encodings, test_labels)

In [8]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(label_encoder.classes_)  # Number of unique allergy classes
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: angathan003 (angathan003-no) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING Serializing object of type dict that is 589912 bytes
wandb: WARNING Serializing object of type dict that is 415152 bytes


Epoch,Training Loss,Validation Loss
1,9.397400,9.301004


TrainOutput(global_step=9552, training_loss=9.285499087130804, metrics={'train_runtime': 1113.9208, 'train_samples_per_second': 17.149, 'train_steps_per_second': 8.575, 'total_flos': 1402192980136704.0, 'train_loss': 9.285499087130804, 'epoch': 1.0})

In [11]:
results = trainer.evaluate()
print(results)

{'eval_loss': 9.301004409790039, 'eval_runtime': 36.2592, 'eval_samples_per_second': 131.718, 'eval_steps_per_second': 65.859, 'epoch': 1.0}
